In [5]:
from lbp.notebooks.scripts.emg_features import *
from scipy.fftpack import fft
from scipy.stats import entropy, skew, kurtosis
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from scipy.integrate import simpson

In [ ]:
df = pd.read_pickle("data/df_for_feature_extraction_with_emg.pkl")

In [5]:
df.shape

(7614602, 150)

In [5]:
df = df.drop(columns=['Frame', 'Sample', 'empty_1','empty_2'])

# feature extraction

In [6]:
def extract_features(df, sampling_rate=1000):
    """
    Extracts features for acc, ang_acc, vel, ang_vel, joint_angle, and EMG data, along with RM and OI values for each repetition.

    Parameters:
    df (pd.DataFrame): Time-series data containing acc, ang_acc, vel, ang_vel, joint_angle, EMG, RM, and OI.
    sampling_rate (int): Sampling rate of the EMG data (default is 1000 Hz).

    Returns:
    pd.DataFrame: A DataFrame containing extracted features with associated RM and OI values.
    """
    feature_list = []
    skipped_reps = []

    # Get the unique participants, sessions, and exercises
    participants = df['Participant'].unique()
    sessions = df['session_id'].unique()
    exercises = df['Marker'].unique()

    # Define EMG columns
    emg_columns_envelope = [col for col in df.columns if 'envelope' in col]
    emg_columns_bandpassed = [col for col in df.columns if 'bandpassed' in col]

    # Loop through participants, sessions, and exercises
    for participant in participants:
        for session in sessions:
            for exercise in exercises:
                exercise_data = df[(df['Participant'] == participant) &
                                   (df['session_id'] == session) &
                                   (df['Marker'] == exercise)]

                reps = exercise_data['Reps'].unique()  # Get unique reps within this exercise

                # Loop through each repetition within the exercise
                for rep in reps:
                    print("Processing Rep:", rep)
                    rep_data = exercise_data[exercise_data['Reps'] == rep]

                    # Ensure there are enough data points for feature extraction
                    if rep_data.empty or len(rep_data) < 2:
                        skipped_reps.append((participant, session, exercise, rep, "Empty or insufficient data"))
                        continue

                    # Extract RM and OI values for the current rep
                    rm_value = rep_data['RM'].iloc[0]
                    oi_value = rep_data['OI'].iloc[0]

                    feature_dict = {
                        'Participant': participant,
                        'session_id': session,
                        'Exercise': exercise,
                        'Rep': rep,
                        'Roland-Morris': rm_value,
                        'Oswestry Index': oi_value
                    }

                    # Add EMG envelope features
                    for col in emg_columns_envelope:
                        if col not in rep_data.columns or rep_data[col].isnull().all():
                            continue
                        try:
                            envelope_col = rep_data[col].dropna().values
                            feature_dict.update({
                                f'{col}_IEMG': CalcIEMG(rep_data, col, sampling_rate),
                                f'{col}_MAV': CalcMAV(rep_data, col),
                                f'{col}_MMAV': CalcMMAV(rep_data, col),
                                f'{col}_SSI': CalcSSI(rep_data, col, sampling_rate),
                                f'{col}_VAR': CalcVAR(rep_data, col),
                                f'{col}_VOrder': CalcVOrder(rep_data, col),
                                f'{col}_RMS': CalcRMS(rep_data, col),
                                f'{col}_WL': CalcWL(rep_data, col),
                                f'{col}_LOG': CalcLOG(rep_data, col),
                                f'{col}_MFL': CalcMFL(rep_data, col),
                                f'{col}_AP': CalcAP(rep_data, col),
                                f'{col}_Min': np.min(envelope_col),
                                f'{col}_Max': np.max(envelope_col),
                                f'{col}_Mean': np.mean(envelope_col),
                                f'{col}_SD': np.std(envelope_col),
                                f'{col}_Skew': skew(envelope_col),
                                f'{col}_Kurtosis': kurtosis(envelope_col),
                            })
                        except Exception as e:
                            skipped_reps.append((participant, session, exercise, rep, f"Error processing EMG envelope '{col}': {e}"))

                    # Add EMG bandpassed spectral features
                    for col in emg_columns_bandpassed:
                        if col not in rep_data.columns or rep_data[col].isnull().all():
                            continue
                        try:
                            data_col = rep_data[col].dropna().values
                            psd = EMG2PSD(data_col, sr=sampling_rate)
                            feature_dict.update({
                                f'{col}_Max_Freq': psd.iloc[psd['Power'].idxmax()]['Frequency'],
                                f'{col}_Twitch_Ratio': CalcTwitchRatio(psd),
                                f'{col}_Twitch_Index': CalcTwitchIndex(psd),
                                f'{col}_Fast_Twitch_Slope': CalcTwitchSlope(psd)[0],
                                f'{col}_Slow_Twitch_Slope': CalcTwitchSlope(psd)[1],
                                f'{col}_Spectral_Centroid': CalcSC(psd),
                                f'{col}_Spectral_Flatness': CalcSF(psd),
                                f'{col}_Spectral_Spread': CalcSS(psd),
                                f'{col}_Spectral_Decrease': CalcSDec(psd),
                                f'{col}_Spectral_Entropy': CalcSEntropy(psd),
                                f'{col}_Spectral_Rolloff': CalcSRoll(psd),
                                f'{col}_Spectral_Bandwidth': CalcSBW(psd, 2),
                            })
                        except Exception as e:
                            skipped_reps.append((participant, session, exercise, rep, f"Error processing EMG bandpassed '{col}': {e}"))

                    # Add motion features
                    motion_columns = [col for col in rep_data.columns if any(sub in col for sub in ['acc', 'ang_acc', 'vel', 'ang_vel'])]
                    for col in motion_columns:
                        series = rep_data[col].dropna().values
                        if len(series) < 2:
                            continue

                        abs_series = np.abs(series)
                        fft_coefficients = fft(series)
                        abs_fft_coefficients = np.abs(fft_coefficients)
                        spectral_entropy = entropy(abs_fft_coefficients)

                        # Number of Mean Crossing Points (NMCP-A)
                        mean_value = np.mean(series)
                        nmcp_a = np.sum((series[:-1] - mean_value) * (series[1:] - mean_value) < 0)

                        # Number of Peaks in Acceleration (NP-A)
                        peaks, _ = find_peaks(series)
                        np_a = len(peaks)

                        # Spectral Arc Length (SPARC)
                        # Normalize the signal
                        normalized_fft = abs_fft_coefficients / np.sum(abs_fft_coefficients)
                        omega = np.linspace(0, np.pi, len(normalized_fft))
                        spectral_arc_length = -simpson(np.sqrt(np.diff(normalized_fft)**2 + np.diff(omega)**2))

                        # Log Dimensionless Jerk in Acceleration (LDLJ-A)
                        time_step = 1 / 1000 
                        jerk = np.diff(series, n=2) / (time_step ** 2)  # Second derivative (jerk)
                        ljerk = np.sum(np.square(jerk)) / ((len(series) - 2) ** 3)
                        ldlj_a = np.log(ljerk + 1e-10)  # Adding a small constant to avoid log(0)

                        # Add motion feature data to feature_dict
                        feature_dict.update({
                            f'{col}_mean': np.mean(series),
                            f'{col}_std': np.std(series, ddof=1),
                            f'{col}_range': np.max(series) - np.min(series),
                            f'{col}_mean_abs': np.mean(abs_series),
                            f'{col}_variance': np.var(series, ddof=1),
                            f'{col}_rms': np.sqrt(np.mean(np.square(series))),
                            f'{col}_skewness': skew(series),
                            f'{col}_kurtosis': kurtosis(series),
                            f'{col}_energy': np.sum(np.square(series)),
                            f'{col}_spectral_entropy': spectral_entropy,
                            f'{col}_abs_max_fft_coeff': np.max(abs_fft_coefficients),
                            f'{col}_abs_min_fft_coeff': np.min(abs_fft_coefficients),
                            f'{col}_sma': np.mean(np.abs(series)),
                            f'{col}_nmcp_a': nmcp_a,
                            f'{col}_np_a': np_a,
                            f'{col}_sparc': spectral_arc_length,
                            f'{col}_ldlj_a': ldlj_a,
                        })

                    # Extract joint angle features
                    joint_angle_columns = [col for col in rep_data.columns if 'joint_angle' in col]
                    for col in joint_angle_columns:
                        series = rep_data[col].dropna().values
                        if len(series) < 2:
                            continue

                        # Add joint angle feature data
                        feature_dict.update({
                            f'{col}_ROM': np.abs(np.max(series) - np.min(series)),
                            f'{col}_mean': np.mean(series),
                            f'{col}_skewness': skew(series),
                        })

                    feature_list.append(feature_dict)

    feature_df = pd.DataFrame(feature_list)
    return feature_df
d = extract_features(df)
print(d.shape)

Processing Rep: 1.0
Processing Rep: 2.0
Processing Rep: 3.0
Processing Rep: 4.0
Processing Rep: 5.0
Processing Rep: 6.0
Processing Rep: 7.0
Processing Rep: 8.0
Processing Rep: 9.0
Processing Rep: 10.0
Processing Rep: 11.0
Processing Rep: 12.0
Processing Rep: 13.0
Processing Rep: 14.0
Processing Rep: 15.0
Processing Rep: 16.0
Processing Rep: 17.0
Processing Rep: 18.0
Processing Rep: 16.0
Processing Rep: 18.0
Processing Rep: 25.0
Processing Rep: 32.0
Processing Rep: 37.0
Processing Rep: 12.0
Processing Rep: 8.0
Processing Rep: 20.0
Processing Rep: 21.0
Processing Rep: 22.0
Processing Rep: 23.0
Processing Rep: 24.0
Processing Rep: 25.0
Processing Rep: 26.0
Processing Rep: 27.0
Processing Rep: 28.0
Processing Rep: 29.0
Processing Rep: 30.0
Processing Rep: 31.0
Processing Rep: 33.0
Processing Rep: 34.0
Processing Rep: 35.0
Processing Rep: 36.0
Processing Rep: 37.0
Processing Rep: 38.0
Processing Rep: 39.0
Processing Rep: 40.0
Processing Rep: 42.0
Processing Rep: 43.0
Processing Rep: 45.0
Pr

# remove correlated ones

In [7]:
def remove_highly_correlated_numerical_keep_one(df, threshold=0.90):
    """
    Removes highly correlated numerical columns from the DataFrame while retaining one feature from each group of correlated features.
    """
    
    # Separate 'Participant' column to keep it intact
    non_numerical_cols = df.select_dtypes(exclude=['number']).columns.tolist()

    numerical_df = df.drop(columns=non_numerical_cols).select_dtypes(include=['number'])
    corr_matrix = numerical_df.corr().abs()

    to_drop = set()
    retained = set()

    for col in corr_matrix.columns:
        if col in to_drop:
            continue
        correlated_features = corr_matrix.columns[corr_matrix[col] > threshold].tolist()
        correlated_features = [f for f in correlated_features if f != col and f not in retained]
        to_drop.update(correlated_features)
        retained.add(col)

    # Exclude target columns 
    excluded_cols = ['Oswestry Index', 'Roland-Morris', 'session_id', 'Reps']
    to_drop = [col for col in to_drop if col not in excluded_cols]
    
    return df.drop(columns=to_drop), list(retained)

filtered_features_df_numerical, retained_features = remove_highly_correlated_numerical_keep_one(d)
print(d.shape)
print(filtered_features_df_numerical.shape)

(970, 2041)
(970, 748)


In [53]:
filtered_features_df_numerical.to_pickle("df_for_predictions.pkl")